# Lever

### **Introduction**

A lever is a simple machine consisting of a rigid bar or rod that pivots around a fixed point, known as the fulcrum. It is used to amplify force, making it easier to lift or move heavy objects. Levers are one of the most fundamental tools in mechanics and have been used for centuries in various applications, from tools and machines to construction and mechanical systems. By changing the position of the fulcrum, the lever allows for a mechanical advantage, which can be used to increase force or distance, depending on the situation.

### **What is it?**

In its most basic form, a lever operates on the principle of moment of force or torque, which is the rotational equivalent of force. When a force is applied at one point on the lever, it creates a rotational force (torque) around the fulcrum. The mechanical advantage provided by a lever depends on the distance from the fulcrum to the point where the force is applied (known as the effort arm) and the distance from the fulcrum to the point where the output force acts (the load arm).

The three main classes of levers are defined by the relative positions of the effort, load, and fulcrum:

1. **First-Class Lever:** The fulcrum is located between the effort and the load. Examples include seesaws or crowbars.
2. **Second-Class Lever:** The load is located between the fulcrum and the effort. Examples include wheelbarrows or nutcrackers.
3. **Third-Class Lever:** The effort is located between the fulcrum and the load. Examples include tweezers or a fishing rod.


### **Example**

Consider a seesaw (a first-class lever) used by two children. One child is sitting at one end of the seesaw, and the other child is sitting at the opposite end. The fulcrum is positioned at the center.

Let’s say the child on one side weighs 30 kg, and the child on the other side weighs 45 kg.

The effort arm is the distance from the fulcrum to the position of the child, and the load arm is the distance from the fulcrum to the other child's position.
If the child with the greater weight sits further from the fulcrum, the lever can balance when the child with lesser weight sits closer. This is an example of using the lever to achieve balance, where the effort exerted by one child is balanced by the load exerted by the other child.

The mechanical advantage of the seesaw would depend on how the children adjust their positions. If the weight is greater on one side, the child would need to move further away from the fulcrum to balance the system. This highlights how the lever can be used to balance forces or move objects with less effort.

In real-world examples, levers are used in various tools, such as hammers (where the handle acts as a lever to increase force applied to the nail), tongs (to grasp objects), and even in the human body, where bones and muscles act like levers.

# Simulation

## Setup

Installing packages (for Google Colab). If this notebook is opened in Google Colab then some packages must be installed to run the code!\
Then importing the scene and plot settings.

In [ ]:
#@title Run to install MuJoCo and `dm_control` for Google Colab

IS_COLAB = 'google.colab' in str(get_ipython())
if IS_COLAB:
    # download the repository
    !git clone https://github.com/commanderxa/alphalabs.git
    from alphalabs.mechanics.setup import install_packages_colab
    install_packages_colab()
    import alphalabs.mechanics.plot
    from alphalabs.mechanics.scene import Scene
else:
    import os, sys
    module_path = os.path.abspath(os.path.join(".."))
    if module_path not in sys.path:
        sys.path.append(module_path)
    # import the scene
    from scene import Scene
    import plot

## Import

Import all required packages to preform simulations. Packages include simulation engine, plotting libraries and other ones necessary for computations.

In [ ]:
%env MUJOCO_GL=egl

import os

# simulation
from dm_control import mjcf, mujoco
from dm_control.mujoco.wrapper.mjbindings import enums

# for video recording
import mediapy

# computations
import numpy as np

## Initial Conditions

In this block constants are defined. They impact the environment, rendering and objects directly.

In [ ]:
# global
viscosity = 0.00000  # Air Resistance

# simulation constants
mass = 10  # [kg]
length = 5  # [m]
force = -50  # [N]
middle = 0.3 # [fraction in [0, 1]]

# rendering
width = 1920
height = 1080
dpi = 600
duration = 10  # (seconds)
framerate = 60  # (Hz)

## Model

### Objects of Interest

This class defines the object of our interest, a `box`. Here we write what is this object (box), what can it do (move, fall, rotate) and also add a camera that follows the object.

In [ ]:
class Box(object):

    def __init__(self, name: str, mass: float, size: float, rgba: list[float]) -> None:
        self.model = mjcf.RootElement(model=name)

        self.box = self.model.worldbody.add("body", name="box", pos=[0, 0, 0])
        self.box_geom = self.box.add(
            "geom",
            name="box_geom",
            type="box",
            size=[size, size, size],
            mass=mass,
            rgba=rgba,
            condim=6,
            friction=[1.0],
        )

        self.model.actuator.add(
            "adhesion",
            name=f"{name}_adhesion",
            body="box",
            ctrlrange=[0, 1],
            gain=10,
        )

        self.fall = self.box.add("joint", name="fall", type="slide", axis=[0, 0, 1])
        self.move = self.box.add("joint", name="move", type="slide", axis=[1, 0, 0])
        self.rotate = self.box.add("joint", name="rotate", type="hinge", axis=[0, 1, 0])

In [ ]:
class Lever(object):

    def __init__(self, length: float, middle: float, mass: float) -> None:
        self.model = mjcf.RootElement(model="lever")

        self.lever_body = self.model.worldbody.add(
            "body", name="lever_body", pos=[0, 0, 0]
        )

        self.leg_length = 0.4
        self.base_leg_1 = self.lever_body.add(
            "geom",
            name="base_leg_1",
            type="box",
            size=[self.leg_length, self.leg_length, 0.02],
            rgba=[1, 1, 0, 1],
            euler=[0, 75, 0],
            pos=[
                np.sin(np.deg2rad(15)) * self.leg_length,
                0,
                np.sin(np.deg2rad(75)) * self.leg_length,
            ],
            conaffinity=2,
            contype=2,
        )
        self.base_leg_2 = self.lever_body.add(
            "geom",
            name="base_leg_2",
            type="box",
            size=[self.leg_length, self.leg_length, 0.02],
            rgba=[1, 1, 0, 1],
            euler=[0, -75, 0],
            pos=[
                np.sin(np.deg2rad(-15)) * self.leg_length,
                0,
                np.sin(np.deg2rad(75)) * self.leg_length,
            ],
            conaffinity=2,
            contype=2,
        )

        self.wrapper = self.lever_body.add(
            "body",
            name="lever_wrapper",
            pos=[0, 0, np.sin(np.deg2rad(75)) * self.leg_length * 2],
        )

        self.half_length = length / 2

        self.line = self.wrapper.add(
            "geom",
            name="line",
            type="box",
            size=[self.half_length, 0.5, 0.05],
            rgba=[1, 0, 0, 1],
            pos=[self.half_length - middle * length, 0, 0],
            condim=6,
            friction=[1.0],
        )

        box_size = 0.7 / 2
        self.box = Box(name="box", mass=mass, size=box_size, rgba=[0, 0, 1, 1])
        self.box_site = self.lever_body.add(
            "site",
            rgba=[0, 0, 0, 0],
            pos=[
                self.half_length - middle * length + self.half_length - box_size,
                0,
                np.sin(np.deg2rad(75)) * self.leg_length * 2 + box_size,
            ],
        )
        self.box_site.attach(self.box.model)

        self.hidden_box = Box(
            name="hidden_box", mass=1, size=box_size, rgba=[0, 0, 0, 0.3]
        )
        self.hidden_site = self.lever_body.add(
            "site",
            rgba=[0, 0, 0, 0],
            pos=[
                self.half_length - middle * length + -self.half_length + box_size,
                0,
                np.sin(np.deg2rad(75)) * self.leg_length * 2 + box_size,
            ],
        )
        self.hidden_site.attach(self.hidden_box.model)

        self.rotate = self.wrapper.add(
            "joint", name="rotate", type="hinge", axis=[0, 1, 0]
        )

### World Model

Collecting everything into one general model.

In [ ]:
class Model(object):

    def __init__(self, length: float) -> None:
        self.model = mjcf.RootElement(model="lever_physics")

        # set render info
        self.model.visual.__getattr__("global").offheight = height
        self.model.visual.__getattr__("global").offwidth = width

        # set the simulation constants
        self.model.option.viscosity = viscosity
        self.model.option.integrator = "RK4"
        self.model.option.timestep = 0.0001

        self.length = length

        # create the environment (ground)
        self.scene = Scene(length=length * 3)
        self.scene_site = self.model.worldbody.add("site", pos=[0, 0, 0])
        self.scene_site.attach(self.scene.model)

        self.lever = Lever(length, middle, mass)
        self.lever_site = self.model.worldbody.add("site", pos=[0, 0, 0])
        self.lever_site.attach(self.lever.model)

        self.camera = self.model.worldbody.add(
            "camera",
            name="front",
            pos=[length / 2 - middle * length, -length, length / 2],
            euler=[75, 0, 0],
        )

## Simulation

Initializing the `physics` of the simulation.

In [ ]:
model = Model(length=length).model
physics = mjcf.Physics.from_mjcf_model(model)

Setting some engine flags. In this case, we want to see all joint that are present.

In [ ]:
scene_option = mujoco.wrapper.core.MjvOption()
scene_option.flags[enums.mjtVisFlag.mjVIS_JOINT] = True

First of all, the environment must be verified by rendering a picture.

In [ ]:
mediapy.show_image(
    physics.render(height, width, camera_id=0, scene_option=scene_option)
)

### Simulation Loop

In [ ]:
frames = []
timevals = []
velocity = []
position = []

physics.bind(model.find_all("actuator")).ctrl = [1, 1]

while physics.data.time < duration:
    physics.named.data.xfrc_applied["lever/hidden_box/box"][2] = (
        force + abs(physics.model.opt.gravity[-1])
    )

    timevals.append(physics.data.time)
    # velocity.append(physics.named.data.qvel["box/move"][0].copy())
    # position.append(physics.named.data.geom_xpos["box/box_geom"][0].copy())

    if len(frames) < physics.data.time * framerate:
        pixels = physics.render(width=width, height=height, camera_id=0)
        frames.append(pixels)

    physics.step()

In [ ]:
mediapy.show_video(frames, fps=framerate)

Save the rendered video

In [ ]:
video_name = f"lever" if IS_COLAB else f"../../output/lever"
mediapy.write_video(video_name + ".mp4", images=frames, fps=framerate)